# An ablation study of the effect of of randomness in MIAs, what components are important and how they contribute to the variance of MIA predictions

### Here are the factors that we will consider in our ablation study:
- Shadow Model Different Initialization: 
- Number of Shadow Models
- Attack Model

### Standard experiment setup in our paper:
- Number of Shadow Models: 10 for Class-NN attack, 20 for reference attack and LiRA, 1 for difficulty calibration attack, 1 for losstraj, 1 for augmentation attack
- Shadow Model Different Initialization: True
- Attack Model: for attacks that uses attack model, we train attack model with seeds according to the attack seed, and this seed would contribute to the data shuffling and batching in the attack model optimization.

We will call this set as *standard*.

### Here are the settings that we will consider in our ablation study:
- *same_shadow_diff_attack*: for the the same attack model, we fix the shadow model used, and try train 6 different attack models based on it to see how much the randomness in attack model contributes to the variance of MIA predictions. For this experiment, we only consider the Class-NN attack and Loss trajectory attack. We will compare this set to *diff_shadow_same_attack*.

- *shadow_init*: study the effect of shadow model initialization. We will compare the results of the same attack model trained on different shadow models with different initialization. We will compare this set to *standard*.

- *diff_number_shadow*: study the effect of number of shadow models. We will compare the results of the same attack model trained on different number of shadow models. We will compare this set to *standard*.
